In [ ]:
from vpython import *
import numpy as np
import os
import csv

# ----------------------------
# Scene setup
scene.background = color.black
scene.width = 1200
scene.height = 700
scene.title = "Moon Orbit in J2000"
scene.autoscale = False 

scale = 1/1000  # km -> VPython units

# ----------------------------
# Earth and Moon
earth = sphere(pos=vector(0,0,0), radius=6371*scale, color=color.blue)
moon = sphere(radius=1737.4*scale, color=vector(0.6,0.6,0.6))

spheres = [earth, moon]

# Labels
label(pos=earth.pos, xoffset=0, yoffset=20, text="Earth", height=10, box=False, line=False)
moon_label = label(pos=moon.pos, xoffset=0, yoffset=20, text="Moon", height=10, box=False, line=False)

# ----------------------------
# load CSV file data
csv_file = "flightdata.csv"
orbit_data = []

try:
    with open(csv_file, 'r') as f:
        lines = f.readlines()

    inside_data = False
    for line in lines:
        line = line.strip()
        if "$$SOE" in line:
            inside_data = True
            continue
        if "$$EOE" in line:
            inside_data = False
            continue
        if inside_data:
            # Split by comma
            parts = line.split(',')
            if len(parts) < 14:
                continue

            # Extract relevant columns
            # Columns: EC=2, QR=3, IN=4, OM=5, W=6, TA=10, A=11 (0-indexed)
            EC = float(parts[2])
            QR = float(parts[3])
            IN_deg = float(parts[4])
            OM_deg = float(parts[5])
            W_deg = float(parts[6])
            TA_deg = float(parts[10])
            A_val = float(parts[11])

            orbit_data.append({
                "EC": EC,
                "QR": QR,
                "IN": np.deg2rad(IN_deg),
                "OM": np.deg2rad(OM_deg),
                "W": np.deg2rad(W_deg),
                "TA": np.deg2rad(TA_deg),
                "A": A_val
            })
    print(f"Loaded {len(orbit_data)} orbit points from CSV.")
except FileNotFoundError:
    print("CSV file not found.")

# ----------------------------
# Generate 3D positions from CSV orbit data
orbit_points = []

for data in orbit_data:
    r = data["A"] * (1 - data["EC"]**2) / (1 + data["EC"] * np.cos(data["TA"]))
    r_j2000 = vector(r * np.cos(data["TA"]), r * np.sin(data["TA"]), 0)

    orbit_points.append(r_j2000 * scale)

# ----------------------------
# Draw the orbit curve
orbit_curve = curve(pos=orbit_points, color=color.yellow, radius=500*scale)

# ----------------------------
# Orbital elements (J2000)
EC = 0.055       # eccentricity
IN = 5.145       # inclination (deg)
OM = 100      # longitude of ascending node (deg)
W  = 318.15      # argument of periapsis (deg)
QR = 363300      # periapsis distance (km)
A  = 384400      # semi-major axis (km)

# Convert degrees to radians
IN_rad = np.deg2rad(IN)
OM_rad = np.deg2rad(OM)
W_rad = np.deg2rad(W)

# ----------------------------
# Camera target
target_center = scene.center

def move_camera_step():
    global target_center
    diff = target_center - scene.center
    if mag(diff) < 1e-3:
        return
    scene.center += diff * 0.1

# ----------------------------
# Ray-sphere intersection for clicks
def ray_intersects_sphere(ray_origin, ray_dir, sphere_center, radius):
    L = np.array([sphere_center.x, sphere_center.y, sphere_center.z]) - np.array([ray_origin.x, ray_origin.y, ray_origin.z])
    tca = np.dot(L, ray_dir)
    d2 = np.dot(L, L) - tca**2
    if d2 > radius**2:
        return False
    thc = np.sqrt(radius**2 - d2)
    t0 = tca - thc
    t1 = tca + thc
    if t0 < 0 and t1 < 0:
        return False
    return True

def on_click(evt):
    global target_center
    ray_origin = scene.camera.pos
    ray_dir = np.array([evt.pos.x - ray_origin.x, evt.pos.y - ray_origin.y, evt.pos.z - ray_origin.z])
    ray_dir = ray_dir / np.linalg.norm(ray_dir)
    for sph in spheres:
        if ray_intersects_sphere(ray_origin, ray_dir, sph.pos, sph.radius):
            target_center = sph.pos
            break

scene.bind('click', on_click)

# ----------------------------
# Earth-centered axes (Z = North Pole)
arrow_length = 50000*scale
arrow(pos=earth.pos, axis=vector(arrow_length,0,0), color=color.red)
label(pos=vector(arrow_length,0,0), text='X', xoffset=10, yoffset=10, height=20, color=color.red)

arrow(pos=earth.pos, axis=vector(0,arrow_length,0), color=color.green)
label(pos=vector(0,arrow_length,0), text='Y', xoffset=10, yoffset=10, height=20, color=color.green)

arrow(pos=earth.pos, axis=vector(0,0,arrow_length), color=color.blue)
label(pos=vector(0,0,arrow_length), text='Z', xoffset=10, yoffset=10, height=20, color=color.blue)

# ----------------------------
# Precompute Moon orbit points in J2000
orbit_points_moon = []
num_points = 200
for nu_deg in np.linspace(0, 360, num_points):
    TA_rad = np.deg2rad(nu_deg)
    r = A * (1 - EC**2) / (1 + EC * np.cos(TA_rad))
    r_perifocal = vector(r*np.cos(TA_rad), r*np.sin(TA_rad), 0)

    # Rotate from orbital plane to J2000
    r1 = rotate(r_perifocal, angle=W_rad, axis=vector(0,0,1))  # argument of periapsis ω
    r2 = rotate(r1, angle=IN_rad, axis=vector(1,0,0))          # inclination i
    r_j2000 = rotate(r2, angle=OM_rad, axis=vector(0,0,1))     # longitude of ascending node Ω

     # longitude of ascending node

    orbit_points_moon.append(r_j2000*scale)

orbit_curve = curve(pos=orbit_points_moon, color=color.gray(0.7), radius=100*scale)

# ----------------------------
# Animation
true_anomaly = np.deg2rad(200)
dTA = 0.00  # radians per frame

# ----------------------------
while True:
    rate(60)
    move_camera_step()

    # Compute Moon position
    r = A * (1 - EC**2) / (1 + EC * np.cos(true_anomaly))
    r_perifocal = vector(r*np.cos(true_anomaly), r*np.sin(true_anomaly), 0)

    # Rotate to J2000 frame
    r1 = rotate(r_perifocal, angle=W_rad, axis=vector(0,0,1))  # argument of periapsis ω
    r2 = rotate(r1, angle=IN_rad, axis=vector(1,0,0))          # inclination i
    r_j2000 = rotate(r2, angle=OM_rad, axis=vector(0,0,1))     # longitude of ascending node Ω



    moon.pos = r_j2000*scale
    moon_label.pos = moon.pos

    true_anomaly += dTA
    if true_anomaly > 2*np.pi:
        true_anomaly -= 2*np.pi


C:\Users\User\Desktop\uni work\radiation spoace stuff\vpython_env\Lib\site-packages\vpython\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


<IPython.core.display.Javascript object>

Loaded 81 orbit points from CSV.
